In [1]:
import io
import os
import glob
import numpy as np
import pandas as pd

In [2]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Mexico','data')

In [3]:
list_dir = [d for d in os.listdir(os.path.join(dir_base,'raw')) if os.path.isdir(os.path.join(os.path.join(dir_base,'raw'), d))]
list_dir = sorted(list_dir)
len_dir = len(list_dir)
print(f'Number of directories: {len_dir}')

Number of directories: 30


In [4]:
id_, nombre_, estado_, municipio_, situacion_, latitud_, longitud_, altitud_ = [],[],[],[],[],[],[],[]
for st in range(len_dir):

    list_files = glob.glob(os.path.join(dir_base,'raw',list_dir[st],'*.txt'))
    list_files = sorted(list_files)
    list_len = len(list_files)
    print(f'{list_dir[st]} has {list_len} stations')

    for nn in range(list_len):

        with open(list_files[nn], 'r', encoding='utf-8') as f:
            lineas = f.readlines()

        info_estacion = {}
        for linea in lineas[10:19]:
            if ':' in linea:
                clave, valor = linea.strip().split(':', 1)
                info_estacion[clave.strip()] = valor.strip()

        id_.append(int(info_estacion['ESTACIÓN']))
        nombre_.append(info_estacion['NOMBRE'])
        estado_.append(info_estacion['ESTADO'])
        municipio_.append(info_estacion['MUNICIPIO'])
        situacion_.append(info_estacion['SITUACIÓN'])
        latitud_.append(float(info_estacion['LATITUD'].replace(' °','')))
        longitud_.append(float(info_estacion['LONGITUD'].replace(' °','')))
        altitud_.append(float(info_estacion['ALTITUD'].replace(' msnm','')))
        
        datos_climaticos = lineas[23:]
        datos_texto = ''.join(datos_climaticos)

        df = pd.read_csv(
            io.StringIO(datos_texto),
            sep=r'\s+',  # usa expresión regular para espacios
            names=['FECHA', 'PRECIP', 'EVAP', 'TMAX', 'TMIN'],
            parse_dates=['FECHA'],
            date_format=lambda x: pd.to_datetime(x, format='%Y-%m-%d', errors='coerce')  # especifica el formato de fecha
        )
        df = df.drop([0, 1]).reset_index(drop=True)
        df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)

        if not os.path.exists(os.path.join(dir_base,'clear',list_dir[st])):
            os.makedirs(os.path.join(dir_base,'clear',list_dir[st]))

        data_out = os.path.join(dir_base,'clear',list_dir[st],list_files[nn].split('/')[-1])
        df.to_csv(data_out,header=True,index=False)

    DATA_INFO = pd.DataFrame({
                            'Id':id_,
                            'Nombre':nombre_,
                            'Estado':estado_,
                            'Municipio':municipio_,
                            'Situacion':situacion_,
                            'Latitud':latitud_,
                            'Longitud':longitud_,
                            'Altitud':altitud_
                            })

Aguas Calientes has 59 stations
Baja California has 66 stations


/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the o

Baja California Sur has 104 stations


/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the o

Campeche has 41 stations
Ciudad de Mexico has 12 stations


...

/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)


Tlaxcala has 24 stations
Veracruz has 72 stations


/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the o

Yucatan has 52 stations
Zacatecas has 68 stations


/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']] = df[['PRECIP', 'EVAP', 'TMAX', 'TMIN']].replace('NULO', np.nan)
/tmp/ipykernel_18215/3148789802.py:40: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the o

In [6]:
dir_out = os.path.join(dir_base,'INFO.csv')
# DATA_INFO.to_csv(dir_out,header=True,index=False)